# Hanker News Dataset Analysis
In this project, we'll work with a data set of submissions to popular technology site Hacker News.

Hacker News is a site started by the startup incubator Y Combinator, where user-submitted stories (known as "posts") are voted and commented upon, similar to reddit. Hacker News is extremely popular in technology and startup circles, and posts that make it to the top of Hacker News' listings can get hundreds of thousands of visitors as a result.

You can find the data set here, but note that it has been reduced from almost 300,000 rows to approximately 20,000 rows by removing all submissions that did not receive any comments, and then randomly sampling from the remaining submissions. 

We'll compare these two types of posts to determine the following:
- Do Ask HN or Show HN receive more comments on average?
- Do posts created at a certain time receive more comments on average?


In [1]:
opened_file = open("HN_posts_year_to_Sep_26_2016.csv")
from csv import reader
read_file = reader(opened_file)
list_file = list(read_file)
hn_header = list_file[0]
hn = list_file[1:]

print(hn_header)
print(hn[1:6])

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']
[['12579005', 'SQLAR  the SQLite Archiver', 'https://www.sqlite.org/sqlar/doc/trunk/README.md', '1', '0', 'blacksqr', '9/26/2016 3:24'], ['12578997', 'What if we just printed a flatscreen television on the side of our boxes?', 'https://medium.com/vanmoof/our-secrets-out-f21c1f03fdc8#.ietxmez43', '1', '0', 'pavel_lishin', '9/26/2016 3:19'], ['12578989', 'algorithmic music', 'http://cacm.acm.org/magazines/2011/7/109891-algorithmic-composition/fulltext', '1', '0', 'poindontcare', '9/26/2016 3:16'], ['12578979', 'How the Data Vault Enables the Next-Gen Data Warehouse and Data Lake', 'https://www.talend.com/blog/2016/05/12/talend-and-Â\x93the-data-vaultÂ\x94', '1', '0', 'markgainor1', '9/26/2016 3:14'], ['12578975', 'Saving the Hassle of Shopping', 'https://blog.menswr.com/2016/09/07/whats-new-with-your-style-feed/', '1', '1', 'bdoux', '9/26/2016 3:13']]


Now that we've removed the headers from hn, we're ready to filter our data. Since we're only concerned with post titles beginning with Ask HN or Show HN, we'll create new lists of lists containing just the data for those titles.

In [2]:
ask_posts = []
show_posts = []
other_posts = []
for row in hn:
    title = row[1]
    title = title.lower()
    if title.startswith("ask hn"):
        ask_posts.append(row)
    elif title.startswith("show hn"):
        show_posts.append(row)
    else:
        other_posts.append(row)
print(len(ask_posts))
print(len(show_posts))
print(len(other_posts))

9139
10158
273822


In the last screen, we separated the "ask posts" and the "show posts" into two list of lists named ask_posts and show_posts. Below are the first five rows in the ask_posts list of lists:

In [3]:
print(hn_header)
ask_posts[:6]
print("\n")
show_posts[:6]

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']




[['12578335',
  'Show HN: Finding puns computationally',
  'http://puns.samueltaylor.org/',
  '2',
  '0',
  'saamm',
  '9/26/2016 0:36'],
 ['12578182',
  'Show HN: A simple library for complicated animations',
  'https://christinecha.github.io/choreographer-js/',
  '1',
  '0',
  'christinecha',
  '9/26/2016 0:01'],
 ['12578098',
  'Show HN: WebGL visualization of DNA sequences',
  'http://grondilu.github.io/dna.html',
  '1',
  '0',
  'grondilu',
  '9/25/2016 23:44'],
 ['12577991',
  'Show HN: Pomodoro-centric, heirarchical project management with ES6 modules',
  'https://github.com/jakebian/zeal',
  '2',
  '0',
  'dbranes',
  '9/25/2016 23:17'],
 ['12577142',
  'Show HN: Jumble  Essays on the go #PaulInYourPocket',
  'https://itunes.apple.com/us/app/jumble-find-startup-essay/id1150939197?ls=1&mt=8',
  '1',
  '1',
  'ryderj',
  '9/25/2016 20:06'],
 ['12576813',
  'Show HN: Learn Japanese Vocab via multiple choice questions',
  'http://japanese.vul.io/',
  '1',
  '1',
  'soulchild37',
  

Compute the average number of comments on ask posts and assign it to avg_ask_comments.

In [4]:
total_ask_comments = 0
for row in ask_posts:
    comments = int(row[4])
    total_ask_comments += comments
print(total_ask_comments)    

94986


In [5]:
num_ask_comments = len(ask_posts)
avg_ask_comments = total_ask_comments / num_ask_comments # avg number of comments for each title
print(avg_ask_comments)

10.393478498741656


Compute the average number of comments on show posts and assign it to avg_show_comments.

In [6]:
total_show_comments = 0
for row in show_posts:
    comments = int(row[4])
    total_show_comments += comments
print(total_show_comments)   

49633


In [7]:
num_show_comments = len(show_posts)
avg_show_comments = total_show_comments / num_show_comments # avg number of comments for each title
print(avg_show_comments)

4.886099625910612


It seems that ask posts receive more average comments. Since ask posts are more likely to receive comments, we'll focus our remaining analysis just on these posts.

Next, we'll determine if ask posts created at a certain time are more likely to attract comments. We'll use the following steps to perform this analysis:

- Calculate the amount of ask posts created in each hour of the day, along with the number of comments received.
- Calculate the average number of comments ask posts receive by hour created.

In [8]:
import datetime as dt
result_list = []
for row in ask_posts:
    created_at = row[6]
    num_comments = int(row[4])
    result_list.append([created_at,num_comments])
counts_by_hour = {}
comments_by_hour = {}
for item in result_list:
    create_date = item[0]
    date_format = "%m/%d/%Y %H:%M"
    create_date = dt.datetime.strptime(create_date,date_format)
    hours = create_date.strftime("%H")
    if hours not in counts_by_hour:
        counts_by_hour[hours] = 1
        comments_by_hour[hours] = item[1]
    else:
        counts_by_hour[hours] += 1
        comments_by_hour[hours] += item[1]
print(counts_by_hour)
print(comments_by_hour)

{'02': 269, '01': 282, '22': 383, '21': 518, '19': 552, '17': 587, '15': 646, '14': 513, '13': 444, '11': 312, '10': 282, '09': 222, '07': 226, '03': 271, '23': 343, '20': 510, '16': 579, '08': 257, '00': 301, '18': 614, '12': 342, '04': 243, '06': 234, '05': 209}
{'02': 2996, '01': 2089, '22': 3372, '21': 4500, '19': 3954, '17': 5547, '15': 18525, '14': 4972, '13': 7245, '11': 2797, '10': 3013, '09': 1477, '07': 1585, '03': 2154, '23': 2297, '20': 4462, '16': 4466, '08': 2362, '00': 2277, '18': 4877, '12': 4234, '04': 2360, '06': 1587, '05': 1838}


We calculated the average number of comments for posts created during each hour of the day, and stored the results in a list of lists named avg_by_hour.

In [9]:
avg_by_hour = []
for hour in comments_by_hour:
    avg_by_hour.append([hour, (comments_by_hour[hour] / counts_by_hour[hour])])
print(avg_by_hour)

[['02', 11.137546468401487], ['01', 7.407801418439717], ['22', 8.804177545691905], ['21', 8.687258687258687], ['19', 7.163043478260869], ['17', 9.449744463373083], ['15', 28.676470588235293], ['14', 9.692007797270955], ['13', 16.31756756756757], ['11', 8.96474358974359], ['10', 10.684397163120567], ['09', 6.653153153153153], ['07', 7.013274336283186], ['03', 7.948339483394834], ['23', 6.696793002915452], ['20', 8.749019607843136], ['16', 7.713298791018998], ['08', 9.190661478599221], ['00', 7.5647840531561465], ['18', 7.94299674267101], ['12', 12.380116959064328], ['04', 9.7119341563786], ['06', 6.782051282051282], ['05', 8.794258373205741]]


Although we now have the results we need, this format makes it hard to identify the hours with the highest values. Let's finish by sorting the list of lists and printing the five highest values in a format that's easier to read.

In [10]:
swap_avg_by_hour = []
for row in avg_by_hour:
    swap_avg_by_hour.append([row[1],row[0]])
print(swap_avg_by_hour)

[[11.137546468401487, '02'], [7.407801418439717, '01'], [8.804177545691905, '22'], [8.687258687258687, '21'], [7.163043478260869, '19'], [9.449744463373083, '17'], [28.676470588235293, '15'], [9.692007797270955, '14'], [16.31756756756757, '13'], [8.96474358974359, '11'], [10.684397163120567, '10'], [6.653153153153153, '09'], [7.013274336283186, '07'], [7.948339483394834, '03'], [6.696793002915452, '23'], [8.749019607843136, '20'], [7.713298791018998, '16'], [9.190661478599221, '08'], [7.5647840531561465, '00'], [7.94299674267101, '18'], [12.380116959064328, '12'], [9.7119341563786, '04'], [6.782051282051282, '06'], [8.794258373205741, '05']]


In [11]:
sorted_swap = sorted(swap_avg_by_hour,reverse=True)
print(sorted_swap[:6])

[[28.676470588235293, '15'], [16.31756756756757, '13'], [12.380116959064328, '12'], [11.137546468401487, '02'], [10.684397163120567, '10'], [9.7119341563786, '04']]


Use the str.format() method to print the hour and average in the following format: "15:00: 38.59 average comments per post".

In [13]:
for row in sorted_swap:
    time_format = "{}: {:.2f} average comments per post"
    swap = dt.datetime.strptime(row[1],"%H")
    swap = swap.strftime("%H:%M")
    swap = time_format.format(swap,row[0])
    print(swap)
   


15:00: 28.68 average comments per post
13:00: 16.32 average comments per post
12:00: 12.38 average comments per post
02:00: 11.14 average comments per post
10:00: 10.68 average comments per post
04:00: 9.71 average comments per post
14:00: 9.69 average comments per post
17:00: 9.45 average comments per post
08:00: 9.19 average comments per post
11:00: 8.96 average comments per post
22:00: 8.80 average comments per post
05:00: 8.79 average comments per post
20:00: 8.75 average comments per post
21:00: 8.69 average comments per post
03:00: 7.95 average comments per post
18:00: 7.94 average comments per post
16:00: 7.71 average comments per post
00:00: 7.56 average comments per post
01:00: 7.41 average comments per post
19:00: 7.16 average comments per post
07:00: 7.01 average comments per post
06:00: 6.78 average comments per post
23:00: 6.70 average comments per post
09:00: 6.65 average comments per post


It seems that 15:00 has the highest number of comments.